In [36]:
import pdfplumber
import pandas as pd
import re
import os


def extract_data_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ''
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + '\n'
    return all_text


def parse_data(text):
    lines = text.split('\n')
    data = []
    for i, line in enumerate(lines[3:]):  # Skip the header row in the PDF text
        try:
            # Remove "merania 2023" if present
            line = line.replace('merania 2023', '').strip()

            # Split the line into parts
            parts = line.split()

            if len(parts) >= 7:
                usek, cesta = parts[:2]
                okres_parts = parts[2:-4]  # Get OKRES parts
                okres = ' '.join(okres_parts)
                survey_parts = parts[-4:]

                # Handle different scenarios based on number of elements
                if len(survey_parts) == 7:
                    T = survey_parts[0] + survey_parts[1]
                    O = survey_parts[2] + survey_parts[3]
                    M = survey_parts[4]
                    S = survey_parts[5] + survey_parts[6]
                elif len(survey_parts) == 6:
                    T = survey_parts[0]
                    O = survey_parts[1] + survey_parts[2]
                    M = survey_parts[3]
                    S = survey_parts[4] + survey_parts[5]
                else:
                    T, O, M, S = survey_parts

                # Convert to integers
                T, O, M, S = map(lambda x: int(
                    x.replace(' ', '')), [T, O, M, S])
                data.append([usek, cesta, okres, T, O, M, S])

        except ValueError as e:
            print(f"Error processing line {i+4}: {line}")
            print(f"Error: {e}")
            # Optional: You can choose to break the loop or continue processing
            # break

    return data



# List of your PDF files
pdf_files = [
    'Data/surveys/csd_tabulka_ba_2022_2023.pdf',
    'Data/surveys/csd_tabulka_bb_2022_2023.pdf',
    'Data/surveys/csd_tabulka_ke_2022_2023.pdf',
    'Data/surveys/csd_tabulka_nr_2022_2023.pdf',
    'Data/surveys/csd_tabulka_po_2022_2023.pdf',
    'Data/surveys/csd_tabulka_tn_2022_2023.pdf',
    'Data/surveys/csd_tabulka_tt_2022_2023.pdf',
    'Data/surveys/csd_tabulka_za_2022_2023.pdf'
]

# List of your PDF files
pdf_files = [
    'Data/surveys/csd_tabulka_tt_2022_2023.pdf'
]

# Initialize an empty DataFrame
all_data = pd.DataFrame()

# Loop through the PDF files
for file in pdf_files:
    text_data = extract_data_from_pdf(file)
    file_data = parse_data(text_data)
    df = pd.DataFrame(file_data, columns=[
                      'ÚSEK', 'CESTA', 'OKRES', 'T', 'O', 'M', 'S'])
    all_data = pd.concat([all_data, df], ignore_index=True)

# Print the combined DataFrame to verify
print(all_data.head(20))

Error processing line 56: ÚSEK CESTA OKRES T O M S Poznámka
Error: invalid literal for int() with base 10: 'O'
Error processing line 109: ÚSEK CESTA OKRES T O M S Poznámka
Error: invalid literal for int() with base 10: 'O'
Error processing line 162: ÚSEK CESTA OKRES T O M S Poznámka
Error: invalid literal for int() with base 10: 'O'
Error processing line 215: ÚSEK CESTA OKRES T O M S Poznámka
Error: invalid literal for int() with base 10: 'O'
Error processing line 268: ÚSEK CESTA OKRES T O M S Poznámka
Error: invalid literal for int() with base 10: 'O'
Error processing line 321: ÚSEK CESTA OKRES T O M S Poznámka
Error: invalid literal for int() with base 10: 'O'
     ÚSEK CESTA                     OKRES    T    O   M    S
0   87036    D1          Trnava 10 734 38  336  115  49  185
1   87040    D1         Hlohovec 7 140 24   76   71  31  286
2   87060    D1         Hlohovec 7 631 24  337   70  32   38
3   87070    D1         Piešťany 7 081 21  392   60  28  533
4   87009    D2         

In [51]:
import pdfplumber
import pandas as pd
import re
import os


def extract_data_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ''
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + '\n'
    return all_text


def parse_data(text):
    lines = text.split('\n')
    data = []
    for line in lines[3:]:  # Skip the header row in the PDF text
        # Remove "merania 2023" if present
        line = line.replace('merania 2023', '').strip()

        # Split the line into parts
        parts = line.split()

        # Check if there are enough parts to parse
        if len(parts) >= 7:
            # ÚSEK is always the first number
            usek = parts[0]

            # CESTA is the second element (number or string like D1, D2, R1, etc.)
            cesta = parts[1]

            # OKRES can be more complex, handle multiple words
            okres_parts = []
            for part in parts[2:]:
                if part.isdigit():
                    break  # Stop when a number is reached
                okres_parts.append(part)
            okres = ' '.join(okres_parts)

            # The rest of the line is the survey data
            survey = ' '.join(parts[len(okres_parts) + 2:])

            data.append([usek, cesta, okres, survey])
    return data



# List of your PDF files
pdf_files = [
    'Data/surveys/csd_tabulka_ba_2022_2023.pdf',
    'Data/surveys/csd_tabulka_bb_2022_2023.pdf',
    'Data/surveys/csd_tabulka_ke_2022_2023.pdf',
    'Data/surveys/csd_tabulka_nr_2022_2023.pdf',
    'Data/surveys/csd_tabulka_po_2022_2023.pdf',
    'Data/surveys/csd_tabulka_tn_2022_2023.pdf',
    'Data/surveys/csd_tabulka_tt_2022_2023.pdf',
    'Data/surveys/csd_tabulka_za_2022_2023.pdf'
]

# Initialize an empty DataFrame
all_data = pd.DataFrame()

# Loop through the PDF files
for file in pdf_files:
    text_data = extract_data_from_pdf(file)
    file_data = parse_data(text_data)
    df = pd.DataFrame(file_data, columns=['ÚSEK', 'CESTA', 'OKRES', 'Survey'])
    all_data = pd.concat([all_data, df], ignore_index=True)

# Print the combined DataFrame to verify
print(all_data.head(20))

     ÚSEK CESTA           OKRES                     Survey
0   87020    D1           Senec    8 968 49 958 140 59 067
1   87021    D1   Bratislava II   17 990 77 187 284 95 461
2   87022    D1   Bratislava II   19 656 80 026 294 99 975
3   87023    D1   Bratislava II  18 284 82 817 305 101 407
4   87024    D1   Bratislava II   12 239 58 640 216 71 095
5   87025    D1   Bratislava II   13 671 69 430 257 83 357
6   87026    D1    Bratislava V   15 154 58 639 215 74 008
7   87028    D1   Bratislava II   11 809 63 728 236 75 773
8   87030    D1           Senec   10 834 47 579 138 58 551
9   87220    D1  Bratislava III    9 524 54 755 203 64 482
10  87010    D2         Malacky    10 541 12 313 66 22 920
11  87011    D2   Bratislava IV   13 233 45 896 219 59 348
12  87012    D2   Bratislava IV   12 646 52 092 250 64 989
13  87013    D2    Bratislava V    11 379 22 173 48 33 600
14  87015    D2    Bratislava V   15 688 78 320 379 94 387
15  87016    D2         Malacky   13 002 19 350 100 32 4

In [53]:
all_data

,ÚSEK,CESTA,OKRES,Survey
0,87020,D1,Senec,8 968 49 958 140 59 067
1,87021,D1,Bratislava II,17 990 77 187 284 95 461
2,87022,D1,Bratislava II,19 656 80 026 294 99 975
3,87023,D1,Bratislava II,18 284 82 817 305 101 407
4,87024,D1,Bratislava II,12 239 58 640 216 71 095
...,...,...,...,...
2810,94920,2338,Liptovský Mikuláš,172 2 380 23 2 575
2811,95211,2341,Liptovský Mikuláš,381 1 835 11 2 227
2812,94040,2358,Liptovský Mikuláš,85 529 10 624
2813,ÚSEK,CESTA,OKRES T O M S Poznámka,


In [54]:
def clean_data(df):
    def is_valid_row(row):
        # Split the survey data by space and check if it's empty
        parts = row['Survey'].split()
        if not parts:
            return False

        try:
            # Convert each part to integer
            numeric_parts = [int(part.replace(' ', '')) for part in parts]

            # Check if the length of numeric_parts is 4 or more
            return len(numeric_parts) >= 4
        except ValueError:
            # If conversion fails, the row is not valid
            return False

    # Apply the is_valid_row function to filter the DataFrame
    cleaned_df = df[df.apply(is_valid_row, axis=1)]

    return cleaned_df


# Apply the cleaning function to all_data
cleaned_all_data = clean_data(all_data)

# Save the DataFrame to a CSV file
# Specify your desired path and file name
csv_file_path = 'Data/surveys/Combined.csv'
cleaned_all_data.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")

Data saved to Data/surveys/Combined.csv


In [24]:
extract_data_from_pdf('Data/surveys/csd_tabulka_tt_2022_2023.pdf')

'CELOŠTÁTNE SČÍTANIE DOPRAVY R. 2022-2023\nTrnavský kraj\nÚSEK CESTA OKRES T O M S Poznámka\n87036 D1 Trnava 10 734 38 336 115 49 185\n87040 D1 Hlohovec 7 140 24 076 71 31 286\n87060 D1 Hlohovec 7 631 24 337 70 32 038\n87070 D1 Piešťany 7 081 21 392 60 28 533\n87009 D2 Senica 10 036 9 245 52 19 332\n80318 R1 Galanta 8 594 30 540 54 39 188 merania 2023\n80796 R1 Trnava 4 419 27 680 102 32 201\n80815 R1 Galanta 11 864 39 084 83 51 031 merania 2023\n80816 R1 Galanta 9 384 34 981 101 44 466 merania 2023\n80817 R1 Galanta 9 960 35 108 79 45 147 merania 2023\n80818 R1 Trnava 8 573 40 280 112 48 966\n80819 R1 Galanta 11 134 34 574 82 45 790 merania 2023\n86550 R1 Trnava 10 267 37 884 69 48 220 merania 2023\n86660 R7 Dunajská Streda 2 048 10 535 39 12 622\n86670 R7 Dunajská Streda 1 842 6 765 25 8 632\n80066 2 Senica 1 574 7 083 27 8 684\n80070 2 Senica 513 3 632 49 4 194\n81862 2 Skalica 1 246 11 843 64 13 153\n81866 2 Skalica 800 6 226 42 7 068\n81867 2 Senica 622 3 737 30 4 389\n82126 13 Du

UNTIL SERVEY IS DONE

In [1]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = 'Data/surveys/Combined.csv'

# Load the CSV file into a DataFrame
df_survey = pd.read_csv(csv_file_path)

df_survey

,ÚSEK,CESTA,OKRES,Survey
0,87020,D1,Senec,8 968 49 958 140 59 067
1,87021,D1,Bratislava II,17 990 77 187 284 95 461
2,87022,D1,Bratislava II,19 656 80 026 294 99 975
3,87023,D1,Bratislava II,18 284 82 817 305 101 407
4,87024,D1,Bratislava II,12 239 58 640 216 71 095
...,...,...,...,...
2752,94911,2337,Liptovský Mikuláš,459 2 798 29 3 286
2753,94920,2338,Liptovský Mikuláš,172 2 380 23 2 575
2754,95211,2341,Liptovský Mikuláš,381 1 835 11 2 227
2755,94040,2358,Liptovský Mikuláš,85 529 10 624


In [2]:
# Define a function to count the number of elements in a list
def count_elements_in_list(survey_str):
    # Split the string by space and count the elements
    elements = survey_str.split()
    return len(elements)


# Apply the function to the "Survey" column and store the result in a new column "n_elems"
df_survey['n_elems'] = df_survey['Survey'].apply(count_elements_in_list)

In [14]:
df_survey

,ÚSEK,CESTA,OKRES,Survey,n_elems
0,87020,D1,Senec,8 968 49 958 140 59 067,7
1,87021,D1,Bratislava II,17 990 77 187 284 95 461,7
2,87022,D1,Bratislava II,19 656 80 026 294 99 975,7
3,87023,D1,Bratislava II,18 284 82 817 305 101 407,7
4,87024,D1,Bratislava II,12 239 58 640 216 71 095,7
...,...,...,...,...,...
2752,94911,2337,Liptovský Mikuláš,459 2 798 29 3 286,6
2753,94920,2338,Liptovský Mikuláš,172 2 380 23 2 575,6
2754,95211,2341,Liptovský Mikuláš,381 1 835 11 2 227,6
2755,94040,2358,Liptovský Mikuláš,85 529 10 624,4


In [5]:
df_survey["n_elems"].value_counts()

n_elems
6    1486
7     909
4     280
5      82
Name: count, dtype: int64

In [15]:
def process_survey(row):
    parts = row['Survey'].split()
    n_elems = len(parts)

    if n_elems == 7:
        # Combine elements (0,1), (2,3), and (5,6)
        T = int(parts[0] + parts[1])
        O = int(parts[2] + parts[3])
        M = int(parts[4])
        S = int(parts[5] + parts[6])
    elif n_elems == 6:
        # Combine elements (1,2) and (4,5)
        T = int(parts[0])
        O = int(parts[1] + parts[2])
        M = int(parts[3])
        S = int(parts[4] + parts[5])
    elif n_elems == 5:
        # Combine elements (3,4)
        T = int(parts[0])
        O = int(parts[1])
        M = int(parts[2])
        S = int(parts[3] + parts[4])
    elif n_elems == 4:
        # Use elements as they are
        T, O, M, S = map(int, parts)
    else:
        # Handle unexpected format
        T = O = M = S = None

    return pd.Series([T, O, M, S])


# Apply the processing function to each row
df_survey[['T', 'O', 'M', 'S']] = df_survey.apply(process_survey, axis=1)

In [27]:
# Update the values for row 2425
df_survey.at[2425, 'T'] = 1551
df_survey.at[2425, 'O'] = 923
df_survey.at[2425, 'M'] = 10
df_survey.at[2425, 'S'] = 2484

In [28]:
# Add a new column "check" to check if S is the sum of T, O, and M
df_survey['check'] = df_survey['S'] - ( df_survey['T'] + \
    df_survey['O'] + df_survey['M'] )

In [29]:
df_survey['check'].value_counts()

check
 0    2723
 1      19
-1      15
Name: count, dtype: int64

In [31]:
selected_columns = ['ÚSEK', 'CESTA', 'OKRES', 'T', 'O', 'M', 'S']
df_survey[selected_columns].to_csv(
    'Data/surveys/Survey.csv', index=False)